## Electric Vehicle Population Data - 2025 
This dataset from the Washington State Open Data Portal contains information about Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered through the Washington State Department of Licensing (DOL). Each record represents an individual vehicle and reflects the growing population of electric vehicles in Washington. 

Access & Use Information
* Source: https://data.wa.gov/Transportation/Electric-Vehicle-Population-Data/f6w7-q2d2/about_data
* Public: This dataset is intended for public access and use.
* License: Open Data Commons Open Database License (ODbL) v1.0

The goal of this notebook is to explore the Washington State EV dataset using Altair, a Python library for creating interactive visualizations. 

### 1. Import data and packages

In [1]:
#!pip install --upgrade altair
#!pip install altair vega_datasets

In [2]:
import pandas as pd
import altair as alt
from vega_datasets import data

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
df = pd.read_csv("Electric_Vehicle_Population_Data_20250825.csv")

### 2. Explore the dataset

In [4]:
print("Number of records: ", len(df))

Number of records:  257635


In [5]:
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJ3E1EB5K,Yakima,Yakima,WA,98901.0,2019,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,220.0,0.0,15.0,347724772,POINT (-120.50729 46.60464),PACIFICORP,5.307700e+10
1,1C4RJXU67R,Kitsap,Port Orchard,WA,98367.0,2024,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21.0,0.0,35.0,272165288,POINT (-122.68471 47.50524),PUGET SOUND ENERGY INC,5.303509e+10
2,KNDCD3LD0N,Snohomish,Lynnwood,WA,98036.0,2022,KIA,NIRO,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,26.0,0.0,32.0,203182584,POINT (-122.29245 47.82557),PUGET SOUND ENERGY INC,5.306105e+10
3,5UXKT0C37H,King,Auburn,WA,98001.0,2017,BMW,X5,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,14.0,0.0,30.0,349010287,POINT (-122.23035 47.3074),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,1N4AZ0CP1D,Skagit,Mount Vernon,WA,98273.0,2013,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,75.0,0.0,40.0,131684150,POINT (-122.33891 48.41644),PUGET SOUND ENERGY INC,5.305795e+10


### 3. Define Questions

**Question 1: What is the most common EV type?**

In [6]:
# Summarize data so it renders quicker
summary = df['Electric Vehicle Type'].value_counts().to_frame(name='Count').reset_index()

In [7]:
# Color palette to use throughout 
categorical = 'set2'
sequential = 'blues'

In [30]:
# Simple bar chart showing number of BEVs and PHEVs in WA
alt.Chart(summary).mark_bar().encode(
    y = 'Electric Vehicle Type',
    x = 'Count', 
    color = alt.Color('Electric Vehicle Type', scale = alt.Scale(range = ['#a6d854', '#8da0cb'])), 
    tooltip = ['Count']
)

alt.Chart(...)

In [27]:
# Now lets view the same data, but this time as a stacked bar chart showing the overall percentange of BEVs vs PHEVs
alt.Chart(summary).transform_joinaggregate(
    TotalEVs='sum(Count)',
).transform_calculate(
    PercentOfWAEVs="datum.Count / datum.TotalEVs"
).mark_bar().encode(
    alt.X('PercentOfWAEVs:Q').axis(format='.0%', title = 'WA State EV Types'), 
    color = alt.Color('Electric Vehicle Type', scale = alt.Scale(range = ['#e78ac3', '#8da0cb'])), 
    tooltip = ['Electric Vehicle Type', alt.Tooltip('PercentOfWAEVs:Q', format='.1%', title='Percentage')]
)

alt.Chart(...)

As you can see, Battery Electric Vehicles are far more common than Plug-in Hybrid Electric Vehicles, accounting for about 80% of the Electric Vehicles in Washington State

## **Question 2: What are the most common makes and models of EVs?**

In [10]:
## Summarize EVS by make
df['Make'].value_counts()

Make
TESLA                     107535
CHEVROLET                  18602
NISSAN                     16274
FORD                       13750
KIA                        12586
BMW                        10656
TOYOTA                     10622
HYUNDAI                     8638
RIVIAN                      7816
VOLVO                       6673
VOLKSWAGEN                  6607
JEEP                        6599
AUDI                        5190
CHRYSLER                    3817
MERCEDES-BENZ               2918
HONDA                       2526
SUBARU                      2473
PORSCHE                     1772
CADILLAC                    1596
POLESTAR                    1453
MAZDA                       1282
MINI                        1202
MITSUBISHI                  1170
LEXUS                       1165
FIAT                         850
DODGE                        759
LUCID                        477
GMC                          472
GENESIS                      433
LINCOLN                      417
ACURA

As you can see, the top brands are Tesla, Chevrolet, and Nissan. Let's look at the top models within each of those brands. 

In [11]:
## Get the top models for Tesla, Chevrolet, and Nissan
tesla = df.loc[df['Make'] == "TESLA"]
tesla_summary = tesla['Model'].value_counts().to_frame(name='Count').reset_index()

chevy = df.loc[df['Make'] == "CHEVROLET"]
chevy_summary = chevy['Model'].value_counts().to_frame(name='Count').reset_index()

nissan = df.loc[df['Make'] == "NISSAN"]
nissan_summary = nissan['Model'].value_counts().to_frame(name='Count').reset_index()

In [12]:
# Now lets visualize the top models for Tesla, Chevrolet, and Nissan in a pie chart
tesla_chart = alt.Chart(tesla_summary).mark_arc().encode(
    theta = 'Count',
    color = alt.Color('Model', scale = alt.Scale(scheme = categorical), legend=alt.Legend(orient='left')), 
    tooltip=['Model:N','Count:Q',alt.Tooltip('percentage:Q', format='.1%', title='Percentage')] # Add the formatted percentage to the tooltip
).transform_joinaggregate(
    total='sum(Count)' # Calculate total sum of 'Count'
).transform_calculate(
    percentage='datum.Count / datum.total' # Calculate percentage for each slice
).properties(title = 'Top Tesla EV Models')

chevy_chart = alt.Chart(chevy_summary).mark_arc().encode(
    theta = 'Count',
    color = alt.Color('Model', scale = alt.Scale(scheme = categorical), legend=alt.Legend(orient='left')), 
    tooltip=['Model:N','Count:Q',alt.Tooltip('percentage:Q', format='.1%', title='Percentage')] # Add the formatted percentage to the tooltip
).transform_joinaggregate(
    total='sum(Count)' # Calculate total sum of 'Count'
).transform_calculate(
    percentage='datum.Count / datum.total' # Calculate percentage for each slice
).properties(title = 'Top Chevrolet EV Models')

nissan_chart = alt.Chart(nissan_summary).mark_arc().encode(
    theta = 'Count',
    color = alt.Color('Model', scale = alt.Scale(scheme = categorical), legend=alt.Legend(orient='left')), 
    tooltip=['Model:N','Count:Q',alt.Tooltip('percentage:Q', format='.1%', title='Percentage')] # Add the formatted percentage to the tooltip
).transform_joinaggregate(
    total='sum(Count)' # Calculate total sum of 'Count'
).transform_calculate(
    percentage='datum.Count / datum.total' # Calculate percentage for each slice
).properties(title = 'Top Nissan EV Models')

In [13]:
# We can combine all three pie charts into a single plot using hconcat. Hovering over each section of the pie will show the model name, count, and relative percentage
alt.hconcat(tesla_chart, chevy_chart, nissan_chart).resolve_scale(color='independent')

alt.HConcatChart(...)

In [14]:
# Now lets visualize the same data but in a bar chart
tesla_bar = alt.Chart(tesla_summary).transform_joinaggregate(
    TotalTeslas='sum(Count)',
).transform_calculate(
    PercentOfTeslass="datum.Count / datum.TotalTeslas"
).mark_bar().encode(
    alt.X('PercentOfTeslass:Q').axis(format='.0%', title = 'Tesla Models'), 
    color = alt.Color('Model', scale = alt.Scale(scheme = categorical), legend=alt.Legend(orient='left')), 
    tooltip = ['Model', alt.Tooltip('PercentOfTeslass:Q', format='.1%', title='Percentage')]
)

nissan_bar = alt.Chart(nissan_summary).transform_joinaggregate(
    TotalTeslas='sum(Count)',
).transform_calculate(
    PercentOfTeslass="datum.Count / datum.TotalTeslas"
).mark_bar().encode(
    alt.X('PercentOfTeslass:Q').axis(format='.0%', title = 'Nissan Models'), 
    color = alt.Color('Model', scale = alt.Scale(scheme = categorical), legend=alt.Legend(orient='left')), 
    tooltip = ['Model', alt.Tooltip('PercentOfTeslass:Q', format='.1%', title='Percentage')]
)

chevy_bar = alt.Chart(chevy_summary).transform_joinaggregate(
    TotalTeslas='sum(Count)',
).transform_calculate(
    PercentOfTeslass="datum.Count / datum.TotalTeslas"
).mark_bar().encode(
    alt.X('PercentOfTeslass:Q').axis(format='.0%', title = 'Chevy Models'), 
    color = alt.Color('Model', scale = alt.Scale(scheme = categorical), legend=alt.Legend(orient='left')), 
    tooltip = ['Model', alt.Tooltip('PercentOfTeslass:Q', format='.1%', title='Percentage')]
)

In [15]:
# Again, we can combine these plots and look at them side by side. Hovering over the bar will show the model name and percentage. 
alt.hconcat(tesla_bar, chevy_bar, nissan_bar).resolve_scale(color='independent')

alt.HConcatChart(...)

As you can see, the top models for Tesla are the Model Y and the Model 3. The top models for Chevrolet are the Bolt EV and the Volt, and the top Nissan model is the Leaf. Finally, lets look at the top models regardless of brand. 

In [16]:
## Summarize EVS by model name
df['Model'].value_counts()

Model
MODEL Y        53560
MODEL 3        37807
LEAF           13971
MODEL S         7911
BOLT EV         7812
               ...  
VF 8               2
S6                 2
SL-CLASS           2
RCV                1
CONTINENTAL        1
Name: count, Length: 179, dtype: int64

In [17]:
## Lets look at the top 5 models, and put everything else in an "other" category
top_5_models = df['Model'].value_counts().nlargest(5)
df['Top_Models'] = df['Model']
df['Top_Models'] = df['Top_Models'].apply(lambda x: x if x in top_5_models else 'Other')
df['Top_Models'].value_counts()

Top_Models
Other      136574
MODEL Y     53560
MODEL 3     37807
LEAF        13971
MODEL S      7911
BOLT EV      7812
Name: count, dtype: int64

In [18]:
# Create a summary df so visualizations render quicker
model_summary = df['Top_Models'].value_counts().to_frame(name='Count').reset_index()

In [19]:
model_summary.columns

Index(['Top_Models', 'Count'], dtype='object')

In [20]:
# Now we can visualize the top models
alt.Chart(model_summary).mark_arc().encode(
    theta = 'Count',
    color = alt.Color('Top_Models', scale = alt.Scale(scheme = categorical), legend = alt.Legend(orient = 'right', title = 'Model')), 
    tooltip = ['Top_Models:N','Count:Q',alt.Tooltip('percentage:Q', format = '.1%', title = 'Percentage')] # Add the formatted percentage to the tooltip
).transform_joinaggregate(
    total = 'sum(Count)' # Calculate total sum of 'Count'
).transform_calculate(
    percentage = 'datum.Count / datum.total' # Calculate percentage for each slice
).properties(title="Top EV Models in WA")

alt.Chart(...)

## **Question 3: Which Washington state counties have the most EVs?**

In [21]:
# summarize EV count by county
# vehicles registered in WA may reside outside of WA, but for the purposes of this analysis, I only want to look at vehicles located in WA
county_summary = df[df['State'] == 'WA']
county_summary = county_summary['County'].value_counts().to_frame(name='Count').reset_index()

As you can see, most EVs are registered in King, Snohomish, and Pierce counties. Let's visualize the number of EVs in each county on a map of Washington state. 

In [22]:
# First we need to pull in a map of the US counties from the vega datasets
# Citation: I used ChatGPT to learn out to how to pull in a map of all US counties
counties = alt.topo_feature(data.us_10m.url, 'counties')

In [23]:
# The TopoJSON maps counties by FIPS code, whereas the EV dataset refers to county by name. 
# I found a list of all the WA state counties and their FIPS code here: https://unicede.air-worldwide.com/unicede/unicede_washington_fips.html
# Then I created a csv file with the state code, county code, combined code, and county name
FIPS_codes = pd.read_csv('WA_FIPS.csv')
FIPS_codes.head()

,FIPS State Code,FIPS County Code,FIPS Code,County Name
0,53,1,53001,Adams
1,53,3,53003,Asotin
2,53,5,53005,Benton
3,53,7,53007,Chelan
4,53,9,53009,Clallam


In [24]:
# Now we can pull in the FIPS Code and add it to the EV dataset
county_summary['FIPS Code'] = county_summary['County'].map(FIPS_codes.set_index("County Name")["FIPS Code"])

In [25]:
# Now we create a choropleth map with the number of EVs by county
# Citation: I used ChatGPT to learn out to how to filter for WA state and create the choropleth map
# The Altair documentation is also useful: https://altair-viz.github.io/altair-tutorial/notebooks/09-Geographic-plots.html

alt.Chart(counties).mark_geoshape(
    stroke="white"
).transform_filter(
    "floor(datum.id / 1000) == 53"
).transform_lookup(
    lookup="id",   
    from_=alt.LookupData(county_summary, "FIPS Code", ["County", "Count"])
).encode(
    color=alt.Color("Count:Q", scale=alt.Scale(scheme = sequential)),
    tooltip=["County:N", "Count:Q"]
).project("mercator").properties(
    width=400,
    height=400,
    title="Electric Vehicles by County in Washington"
)

alt.Chart(...)

As you can see, the majority of EVs are located in King county, followed up by Snohomish and Pierce counties. This probably makes sense given that EV chargers tend to be located in more populated areas. 